In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from tensorflow.keras.models import Sequential

from sklearn.model_selection import KFold
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

SCENE="1"
CSV_FILE_PATH = "./audioset_3sec_escena"+SCENE+".csv"  # path of csv file
DATA_PATH = "./audioset_3sec_escena"+SCENE+"/" # path to folde

In [2]:
df = pd.read_csv(CSV_FILE_PATH)
df


,name,target,category
0,vacuum00.2.wav,1,aspirador
1,vacuum01.2.wav,1,aspirador
2,vacuum010.2.wav,1,aspirador
3,vacuum011.2.wav,1,aspirador
4,vacuum012.2.wav,1,aspirador
...,...,...,...
616,doorbell058.2.wav,12,timbre
617,doorbell06.2.wav,12,timbre
618,doorbell07.2.wav,12,timbre
619,doorbell08.2.wav,12,timbre


In [3]:
sample_df = df.drop_duplicates(subset=['target'])
sample_df

,name,target,category
0,vacuum00.2.wav,1,aspirador
52,toilet_flush00.2.wav,2,cisterna
94,conversation00.2.wav,3,conversacion
139,dishes_pots_and_pans00.2.wav,4,cubiertos_sartenes
191,alarm_clock00.2.wav,5,despertador
244,water00.2.wav,6,ducha
299,water_tap00.2.wav,7,grifo_cocina
369,printer00.2.wav,8,impresora
420,microwave00.2.wav,9,microondas
470,door00.2.wav,10,puerta


In [4]:
X , y = [] , []
for data in tqdm(df.iterrows()):
  data  
  sig , sr = librosa.load(DATA_PATH+data[1][0])
  sig.shape
  mel_spec_ = librosa.feature.melspectrogram(y=sig , sr=sr ,  n_fft=2048, hop_length=512)
  mel_spec_ = librosa.power_to_db(mel_spec_, ref=np.max)  #visualizing mel_spectrogram directly gives black image. So, coverting from power_to_db is required
    

  mel_spec_.shape
  size=int(mel_spec_.shape[1]/2)
  size
  mel_spec_=mel_spec_[:,size-65:size+65]
  data[1][1] 
  mel_spec_.shape
  

  if(mel_spec_.shape != (128, 130)):
        print("bad audio sample size")
        continue
  X.append(mel_spec_)
  y.append(data[1][1])

    
from collections.abc import Sequence


def get_shape(lst, shape=()):
    """
    returns the shape of nested lists similarly to numpy's shape.

    :param lst: the nested list
    :param shape: the shape up to the current recursion depth
    :return: the shape including the current depth
            (finally this will be the full depth)
    """

    if not isinstance(lst, Sequence):
        # base case
        return shape

    # peek ahead and assure all lists in the next depth
    # have the same length
    if isinstance(lst[0], Sequence):
        l = len(lst[0])
        if not all(len(item) == l for item in lst):
            msg = 'not all lists have the same length'
            raise ValueError(msg)

    shape += (len(lst), )
    
    # recurse
    shape = get_shape(lst[0], shape)

    return shape


print(get_shape(X))
print(get_shape(y))



0it [00:00, ?it/s]

(0,
 name        vacuum00.2.wav
 target                   1
 category         aspirador
 Name: 0, dtype: object)

(220500,)

(128, 431)

215

...

50

12

(128, 15)

621it [03:08,  3.29it/s]

bad audio sample size
(617,)
(617,)


In [5]:
print(y)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [6]:

# convert list to numpy array
X = np.array(X) 
y = np.array(y)

#one-hot encoding the target
y = tf.keras.utils.to_categorical(y-1 , num_classes=12)

# our tensorflow model takes input as (no_of_sample , height , width , channel).
# here X has dimension (no_of_sample , height , width).
# So, the below code will reshape it to (no_of_sample , height , width , 1).
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
X.shape
y.shape


(617, 128, 130, 1)

(617, 12)

In [7]:
x_train , x_val , y_train , y_val = train_test_split(X , y ,test_size=0.1, random_state=1)


print(np.sum(y_train, axis=0))
print(np.sum(y_val, axis=0))


INPUTSHAPE = (128,130,1)
NUM_CLASSES=12

model =  models.Sequential([
                          layers.Conv2D(16 , (3,3),activation = 'relu',padding='valid', input_shape = INPUTSHAPE),
                          layers.Conv2D(16, (3,3), activation='relu',padding='valid'),

                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(64, (3,3), activation='relu',padding='valid'),

                          layers.Conv2D(128, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(256, (3,3), activation='relu',padding='valid'),

                          layers.GlobalAveragePooling2D(),


                          layers.Dense(1024 , activation = 'relu'),
                          layers.Dense(1024 , activation = 'relu'),
    
                          layers.Dense(NUM_CLASSES , activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'acc')

model.summary()

[46. 34. 44. 45. 50. 49. 61. 49. 43. 55. 44. 35.]
[6. 8. 1. 7. 3. 6. 9. 2. 6. 4. 4. 6.]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 128, 16)      160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 126, 16)      2320      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 124, 32)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 122, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 118, 120, 128)     73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 116, 118, 256)     295168    
__________________________________

In [8]:
LOGDIR = "logs"
CPKT = "cpkt/"

#this callback is used to prevent overfitting.
callback_1 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

#this checkpoint saves the best weights of model at every epoch
callback_2 = tf.keras.callbacks.ModelCheckpoint(
    CPKT, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None
)

#this is for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOGDIR)


history = model.fit(x_train,y_train ,
            validation_data=(x_val,y_val),
            epochs=40)
# Guardar el Modelo
model.save("model_audio_Escena"+SCENE+".h5")

Epoch 1/40
18/18 [==============================] - 85s 5s/step - loss: 2.6250 - acc: 0.0942 - val_loss: 2.4829 - val_acc: 0.1452
Epoch 2/40
18/18 [==============================] - 76s 4s/step - loss: 2.4815 - acc: 0.1076 - val_loss: 2.4860 - val_acc: 0.1774
Epoch 3/40
18/18 [==============================] - 79s 4s/step - loss: 2.4760 - acc: 0.0952 - val_loss: 2.4805 - val_acc: 0.1452
Epoch 4/40
18/18 [==============================] - 79s 4s/step - loss: 2.4754 - acc: 0.1001 - val_loss: 2.4763 - val_acc: 0.1452
Epoch 5/40
18/18 [==============================] - 79s 4s/step - loss: 2.4477 - acc: 0.1413 - val_loss: 2.4324 - val_acc: 0.1452
Epoch 6/40
18/18 [==============================] - 76s 4s/step - loss: 2.4472 - acc: 0.1178 - val_loss: 2.4614 - val_acc: 0.1129
Epoch 7/40
18/18 [==============================] - 76s 4s/step - loss: 2.4301 - acc: 0.1534 - val_loss: 2.3895 - val_acc: 0.1290
Epoch 8/40
18/18 [==============================] - 78s 4s/step - loss: 2.3971 - acc: 0.14

In [9]:
#Confution Matrix

y_pred = np.argmax(model.predict(x_val),axis=1)
print('Confusion Matrix')
confusionMatrix = confusion_matrix(y_pred, np.argmax(y_val,axis=1))
print(confusionMatrix)

Confusion Matrix
[[2 1 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 2 0 1 2 1 0 0 1 0]
 [1 0 0 1 2 0 0 0 0 1 0 1]
 [1 0 0 0 0 2 1 0 4 0 0 0]
 [0 6 0 1 0 2 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [2 1 0 2 0 0 3 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 3 4]]


In [10]:
from sklearn.metrics import classification_report
classification = classification_report(y_pred, np.argmax(y_val,axis=1),output_dict=False)
print(classification)

classification = classification_report(y_pred, np.argmax(y_val,axis=1),output_dict=True)
dataframe = pd.DataFrame(classification).transpose()


              precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.29      0.25      0.27         8
           4       0.67      0.33      0.44         6
           5       0.33      0.25      0.29         8
           6       0.33      0.25      0.29        12
           7       0.00      0.00      0.00         0
           8       0.17      0.09      0.12        11
           9       0.50      0.67      0.57         3
          10       0.00      0.00      0.00         1
          11       0.67      0.57      0.62         7

    accuracy                           0.29        62
   macro avg       0.27      0.23      0.25        62
weighted avg       0.36      0.29      0.32        62



c:\users\uja\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\uja\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\uja\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)